In [ ]:
#Setup 
import inspect
import netron
import numpy as np
from finn.util.basic import make_build_dir
from IPython.display import IFrame
from pkgutil import get_data
import onnx
import onnx.numpy_helper as nph
import torch
from finn.custom_op.xnorpopcount import xnorpopcountmatmul
from finn.core.modelwrapper import ModelWrapper

from finn.util.test import get_test_model_trained
def showSrc(what):
    print("".join(inspect.getsourcelines(what)[0]))
    
def showInNetron(model_filename):
    netron.start(model_filename, port=8081, host="0.0.0.0")
    return IFrame(src="http://0.0.0.0:8081/", width="100%", height=400)
    

fc = get_test_model_trained("TFC", 1, 1)
raw_i = get_data("finn", "data/onnx/mnist-conv/test_data_set_0/input_0.pb")
#Compare output, +-5% different should be good
input_tensor = onnx.load_tensor_from_string(raw_i)
input_brevitas = torch.from_numpy(nph.to_array(input_tensor)).float()
output_golden = fc.forward(input_brevitas).detach().numpy()
output_golden

    
build_dir = "/workspace/finn"



In [ ]:
#python simulation, before HLS layers

input_dict = {"global_in": nph.to_array(input_tensor)}

model_for_sim = ModelWrapper(build_dir+"/tfc_w1a1_ready_for_hls_conversion.onnx")

import finn.core.onnx_exec as oxe
output_dict = oxe.execute_onnx(model_for_sim, input_dict)
output_pysim = output_dict[list(output_dict.keys())[0]]


if np.isclose(output_pysim, output_golden, atol=1e-3).all():
    print("Results are the same!")
else:
    print("The results are not the same!")

In [ ]:
#c++ simulation, after hls layers
from finn.transformation.fpgadataflow.codegen_npysim import CodeGen_npysim
from finn.transformation.fpgadataflow.compile import Compile

model_for_npysim = model_for_npysim.transform(CodeGen_npysim())
model_for_npysim = model_for_npysim.transform(Compile())

model_for_npysim.save(build_dir+"/tfc_w1_a1_for_npysim.onnx")
showInNetron(build_dir+"/tfc_w1_a1_for_npysim.onnx")

In [ ]:
#Emulation (rtlsim) using PyVerilator, after IP generation